In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from pathlib import Path

# ---------------------------
# Mann-Kendall Test Function
# ---------------------------
def mann_kendall_test(series, alpha=0.05):
    """
    Performs the Mann-Kendall test for monotonic trend.
    This version does not include tie correction.
    """
    x = np.array(series)
    x = x[~np.isnan(x)]  # Remove NaN values
    n = len(x)

    if n < 3:
        # Not enough data to perform the test
        return np.nan, np.nan, np.nan, np.nan, "Not Enough Data"

    S = 0
    for k in range(n - 1):
        for j in range(k + 1, n):
            if x[j] > x[k]:
                S += 1
            elif x[j] < x[k]:
                S -= 1

    # Note: This variance calculation does not account for ties in the data
    var_S = (n * (n - 1) * (2*n + 5)) / 18

    if var_S == 0:
        # Cannot compute Z if variance is zero
        return S, var_S, np.nan, np.nan, "Variance is Zero"

    if S > 0:
        Z = (S - 1) / np.sqrt(var_S)
    elif S < 0:
        Z = (S + 1) / np.sqrt(var_S)
    else:
        Z = 0

    p = 2 * (1 - norm.cdf(abs(Z)))
    Z_crit = norm.ppf(1 - alpha/2)

    if abs(Z) > Z_crit:
        trend = "Increasing" if Z > 0 else "Decreasing"
    else:
        trend = "No Significant Trend"

    return S, var_S, Z, p, trend

# ---------------------------
# Process Your Excel File
# ---------------------------
file = Path(r"D:\climate change\monthly_averages.xlsx")
sheets = ["Rainfall_monthly", "Tmax_monthly", "Tmin_monthly"]  # modify if needed

results = []

print(f"Starting Mann-Kendall analysis on: {file.name}")

for sheet in sheets:
    print(f"\n📂 Processing Sheet: {sheet}")
    try:
        df = pd.read_excel(file, sheet_name=sheet, index_col=0)
        df = df.drop(columns=["Lon", "Lat"], errors="ignore")

        for station in df.columns:
            S, varS, Z, p, trend = mann_kendall_test(df[station])

            # --- MODIFICATION HERE ---
            # Round all numeric results to 2 decimal points
            results.append({
                "Sheet": sheet,
                "Station": station,
                "S": round(S, 2),
                "Var(S)": round(varS, 2),
                "Z": round(Z, 2),
                "p-value": round(p, 2),
                "Trend": trend
            })
        print(f"  ...processed {len(df.columns)} stations.")

    except Exception as e:
        print(f"  ...Error processing sheet {sheet}: {e}")

# ---------------------------
# Save Output
# ---------------------------
output = Path(r"D:\climate change\MK_results.csv")
pd.DataFrame(results).to_csv(output, index=False)

print(f"\n✅ Mann-Kendall results saved to:\n{output}")


Starting Mann-Kendall analysis on: monthly_averages.xlsx

📂 Processing Sheet: Rainfall_monthly
  ...processed 23 stations.

📂 Processing Sheet: Tmax_monthly
  ...processed 23 stations.

📂 Processing Sheet: Tmin_monthly
  ...processed 23 stations.

✅ Mann-Kendall results saved to:
D:\climate change\MK_results.csv
